# 查看数据形式，并读取数据

In [34]:
import os
import random
from scipy import misc

train_path = './train/'
test_path = './test1/'

train_names = os.listdir(train_path)
test_names = os.listdir(test_path)

for i in range(5):
    sample = random.choice(train_names)
    image = misc.imread(train_path+sample)
    print('name: {}'.format(sample))
    print('shape: {}'.format(image.shape))
    print('type: {}\n'.format(image.dtype))
    

name: cat.8705.jpg
shape: (226L, 275L, 3L)
type: uint8

name: dog.7364.jpg
shape: (375L, 499L, 3L)
type: uint8

name: dog.6690.jpg
shape: (195L, 259L, 3L)
type: uint8

name: cat.2176.jpg
shape: (412L, 323L, 3L)
type: uint8

name: cat.8304.jpg
shape: (374L, 500L, 3L)
type: uint8

